In [1]:
import yaml
import ml_collections

In [2]:
config_path = '/root/projects/deeponet/deeponet/config.yaml'

In [3]:
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)
    cfg = ml_collections.ConfigDict(cfg)

In [4]:
cfg.model

activation: tanh
branch_net_1:
  hidden_units:
  - 512
  - 512
  - 512
  - 512
  - 512
  input_key: sigma_a
  output_key: branch_1
  type: modified_mlp
branch_net_2:
  hidden_units:
  - 512
  - 512
  - 512
  - 512
  - 512
  input_key: sigma_t
  output_key: branch_2
  type: modified_mlp
branch_net_3:
  hidden_units:
  - 512
  - 512
  - 512
  - 512
  - 512
  input_key: boundary
  output_key: branch_3
  type: modified_mlp
latent_size: 512
trunk_net:
  hidden_units:
  - 512
  - 512
  - 512
  - 512
  - 512
  input_key: phase_coords
  output_key: trunk
  type: mlp

In [5]:
for d in cfg.model.values():
    print(d == cfg.model.trunk_net, isinstance(d, ml_collections.ConfigDict))
    if isinstance(d, ml_collections.ConfigDict):
        print(d.get('input_key'))

False True
sigma
False True
boundary
False False
True True
phase_coords


In [6]:
cfg.model.trunk_net.get('input_key', [])

'phase_coords'

In [7]:
trunk_key = cfg.model.trunk_net.get('input_key')
branch_keys = [d.get('input_key', []) for d in cfg.model.values() if isinstance(d, ml_collections.ConfigDict) and d != cfg.model.trunk_net]

In [8]:
import numpy as np
DATA_PATH = "/root/projects/deeponet/data/g0.1-sigma_a3-sigma_t6.npz"
BRANCH_KEYS = ['sigma', 'boundary']
TRUNK_KEYS = ['phase_coords']

np_data = dict(np.load(DATA_PATH, allow_pickle=True))

In [9]:
def _generate_input_keys(cfg, np_data):
    trunk_key = cfg.model.trunk_net.get('input_key')
    branch_keys = [d.get('input_key', []) for d in cfg.model.values() if isinstance(d, ml_collections.ConfigDict) and d != cfg.model.trunk_net]

    return trunk_key, branch_keys
    

In [6]:
import torch.nn as nn
def _get_activation():
    act = cfg.model.get("activation", "relu")
    if act == "tanh":
        return nn.Tanh
    elif act == "relu":
        return nn.ReLU